# Integrating an Observation Tool in an Agent

> **Learning Outcomes:**
> - Create an observation tool with AutoGen.
> - Integrate the observation tool with an agent.
> - Understand the AutoGen Agent flow.

## Introduction

In this lab, we will implement an observation tool that will allow the agent to observe their environment. This tool won't have any affect on the world outside of the agent, but will allow the agent to gather information about that world. This is a useful capacity for an agent to have, as it allows the agent to make more informed decisions.

### AutoGen Framework
[AutoGen](https://microsoft.github.io/autogen/stable/index.html) is "a framework for building AI agents and applications" that is developed by Microsoft. It is designed to be a flexible and extensible framework that can be used to build a wide range of AI applications.

We will be working with the `autogen-agentchat` component which is designed for building conversational single and multi-agent systems. This component is built on top of the `autogen` framework and provides a simple way to build agents that can communicate with each other.


### The Scenario
Our agent will be tasked with converting Wikipedia pages into a timeline. The agent will be able to observe the Wikipedia page and extract the information needed to create the timeline. The agent will then be able to use this information to create a timeline of the events described in the Wikipedia page. If the agent needs additional information, it can perform more observations to gather that information.


## Getting Started
Let's start by installing the required libraries and setting the OpenAI API key. The OpenAI API key is required to access the OpenAI models.

Run the following cells to install the required libraries and set the OpenAI API key.


In [ ]:
%pip install --upgrade pip setuptools wheel
%pip install tiktoken --only-binary=:all:

In [ ]:
%pip install -qU \
    openai==1.95.* \
    autogen-agentchat==0.6.* \
    autogen-ext[openai]==0.6.* \
    nlpia2-wikipedia==1.5.*

import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

## Core

### Step 1 - Write Query Function

We will use the `nlpia2-wikipedia` library to work with Wikipedia. This library provides three functions, which are demonstrated in the cells below.

In [ ]:
import wikipedia

results = wikipedia.search("New York City") # List of titles
results

In [ ]:
# Summary of the first title
summary = wikipedia.summary(results[0])
summary

In [ ]:
#Full page of the first title
page = wikipedia.page(results[0])
print('Title:', page.title)
print('URL:', page.url)
print('Content:', page.content[:1000])
print('Summary:', page.summary[:1000])
print('Links:', page.links)

Most Agentic libraries, including AutoGen, allow tools to be written as functions. These functions are then converted to tools by the library for easy integration with the agent. While the exact method of writing these functions may vary depending on the library, the general process is the same.

1. Write a function that takes the necessary inputs and returns the desired output.
2. Add type hints to the function to specify the types of the inputs and outputs.
3. Create a description of the function.


```python
def hello_world(name: str) -> str:
    return f"Hello, {name}!"
```

In the cell below, write a function called `get_wikipedia_page` that takes a string as input and returns the content of the Wikipedia page for that string.

In [ ]:
# YOUR CODE HERE


A function on it's own is not a tool. It needs to be converted to a tool using the `autogen` library. This is done by using the `FunctionTool` class. This class takes the function and a description of the function as input and creates a tool that can be used by the agent.

```python
tool = FunctionTool(hello_world, description="Greets the person by name.")
await tool.run_json({"title": "John"})
```

In the cell below, create a `FunctionTool` for the your function and test it by running the tool directly. Save the tool to the variable `page_tool`.

In [ ]:
from autogen_core.tools import FunctionTool
from autogen_core import CancellationToken

# YOUR CODE HERE


### Step 2 - Creating an Agent

Now that you have a tool, we need to integrate it with the agent. AutoGen's `AssistantAgent` class has the built-in capability to use a tool and respond based on it's output. However, it can only use one tool, once per task. To allow the the agent to use the tool multiple times, we can place it in a round-robin group chat by itself. This will allow the agent to continue working until it is complete.

Run the cell below to see how the agent uses the tool.

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.ui import Console
import textwrap

model_client = OpenAIChatCompletionClient(model="gpt-4.1-mini")
agent = AssistantAgent(
    name="timeline_creator",
    model_client=model_client,
    tools=[page_tool],
    system_message=textwrap.dedent("""
        You are an AI assistant that can help users create timelines of events based on the content of Wikipedia pages.
        When the user requests a timeline, perform the following tasks:

        1. Query Wikipedia for the content of the page.
        2. Use the content to extract dates.
        3. Use the extracted dates for form a timeline.

        Reply with TERMINATE when the task has been completed.
    """.strip()),
)

team = RoundRobinGroupChat([agent], termination_condition=TextMentionTermination("TERMINATE"))


await Console(team.run_stream(task="Create a timeline of Python."))

### Step 3 - Extending the Capabilities

Now that we have a tool that can get the content of a Wikipedia page, we can extend the agent's capabilities by adding more tools. The Wikipedia library can also search for pages based on a query.

In the cell below, write a function called `search_wikipedia` that takes a string as input and returns the list of Wikipedia pages that match that string. Use that function to create a tool called `search_tool` and test it by running the tool directly.

In [ ]:
# YOUR CODE HERE


Now that we have a tool that can search for Wikipedia pages, we can integrate it with the agent. In the cell below, create a new `AssistantAgent` and round-robin group chat that uses the `search_tool` and `page_tool` tools. Test the agent by running the cell.

In [ ]:
# YOUR CODE HERE


Great Job!

Now you have an agent that can search for and read Wikipedia pages. You can extend the agent's capabilities by adding more tools to it. In the challenges below, we'll do just that.

## Bonus Challenge 1 - File System Agent

Create a set of tools that allows an agent to search for and read files on the local file system. Integrate these tools with the agent and test them.

In [ ]:
# YOUR CODE HERE

## Bonus Challenge 2 - API Access Tools

A common need is for an agent to access an API to gather information. [public-apis](https://github.com/public-apis/public-apis) is a list of public APIs that can be easily integrated into AI Agents. Select an API from the list and create a set of tools that allow an agent to access that API. Integrate these tools with the agent and test them.

If you need a suggestion, try the [US National Weather Service API](https://www.weather.gov/documentation/services-web-api).

In [ ]:
# YOUR CODE HERE

# Conclusion

In this lab, we created an observation tool that allows an agent to gather information about the world around it. We integrated this tool with an agent and used it to create a timeline of events from a Wikipedia page. We then extended the agent's capabilities by adding more tools to it.

In the challenges, we created a set of tools that allow an agent to search for and read files on the local file system and access an API to gather information. We integrated these tools with the agent and tested them.

Integrating tools with agents is essential for building AI applications that can interact with the world around them. By creating tools that allow agents to observe and interact with their environment, we can build agents that can perform a wide range of tasks.

